In [ ]:
import pandas as pd
import numpy as np

In [ ]:
season_dat = pd.read_csv('data/DataFiles/RegularSeasonCompactResults.csv')
season_dat.shape

In [ ]:
tourney_dat = pd.read_csv('data/DataFiles/NCAATourneySeeds.csv')
tourney_dat.shape

In [4]:
tourney_dat.head()

## Limit data to 2004 onwards. 

In [5]:
szn_dat = season_dat[season_dat['Season']>=2004]
szn_dat.shape

(82888, 8)

In [6]:
t_dat = tourney_dat[tourney_dat['Season']>=2004]
t_dat['Seed'] = t_dat['Seed'].str.replace('[a-zA-Z]', '').astype('int64')

/Users/meyappansubbaiah/Documents/Github/March_Madness_KG/.env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [13]:
tourney_teams = t_dat[['Season','TeamID']]

In [14]:
### Identify tourney teams

In [15]:
# confirm number of tourney teams per year
tourney_teams.groupby('Season').agg('count')

,TeamID
Season,
2004,65
2005,65
2006,65
2007,65
2008,65
2009,65
2010,65
2011,68
2012,68


In [16]:
## Extract games for only teams that were in the tourney that year.  

In [17]:
szn_tourney_teams_part_a = pd.merge(szn_dat,tourney_teams,how='inner',
                                   left_on=['Season', 'WTeamID'], 
                                   right_on=['Season', 'TeamID'])
szn_tourney_teams_part_b = pd.merge(szn_dat,tourney_teams,how='inner',
                                   left_on=['Season', 'LTeamID'], 
                                   right_on=['Season', 'TeamID'])

In [18]:
szn_tourney_teams = pd.concat([szn_tourney_teams_part_a,szn_tourney_teams_part_b])
szn_tourney_teams = szn_tourney_teams.drop('TeamID',axis=1)
szn_tourney_teams = szn_tourney_teams.fillna(0)
szn_tourney_teams.shape

(33925, 8)

In [19]:
szn_tourney_teams = szn_tourney_teams.drop_duplicates()
szn_tourney_teams.shape

(29215, 8)

In [20]:
def w_year_df(row):
    year_ext_df = tourney_teams[tourney_teams['Season'] == row['Season']]
    return (row['WTeamID'] in year_ext_df['TeamID'].tolist())

In [21]:
def l_year_df(row):
    year_ext_df = tourney_teams[tourney_teams['Season'] == row['Season']]
    return (row['LTeamID'] in year_ext_df['TeamID'].tolist())

In [22]:
szn_tourney_teams['WTID_in_tourney'] = szn_tourney_teams.apply(w_year_df,axis=1)

In [23]:
szn_tourney_teams['LTID_in_tourney'] = szn_tourney_teams.apply(l_year_df,axis=1)

In [24]:
szn_tourney_teams.to_csv('data/Final/network_adj_szn.csv',index=False)